In [ ]:
import numpy as np

class TOPSIS:
    @staticmethod
    def calculate_topsis(updated_df, all_weighted_cols):
        ideal_best = updated_df[all_weighted_cols].max()
        ideal_worst = updated_df[all_weighted_cols].min()

        updated_df['S_i+'] = np.sqrt(((updated_df[all_weighted_cols] - ideal_best) ** 2).sum(axis=1))
        updated_df['S_i-'] = np.sqrt(((updated_df[all_weighted_cols] - ideal_worst) ** 2).sum(axis=1))
        updated_df['TOPSIS_score'] = updated_df['S_i-'] / (updated_df['S_i+'] + updated_df['S_i-'])

        updated_df.sort_values(by='TOPSIS_score', ascending=False, inplace=True)
        updated_df[['player_name', 'TOPSIS_score']].to_csv('output_ranking_TOPSIS.csv', index=False)
        return updated_df[['player_name', 'TOPSIS_score']]